In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install --upgrade git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier

  Cloning https://github.com/terrier-org/pyterrier.git to c:\users\halim\appdata\local\temp\pip-install-4c0e9jch\python-terrier_7a7b1519212f45a2a7404860bc059f3e
  Resolved https://github.com/terrier-org/pyterrier.git to commit a17f9b7ed1782010b4680457874f220b65fa2594
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/terrier-org/pyterrier.git 'C:\Users\halim\AppData\Local\Temp\pip-install-4c0e9jch\python-terrier_7a7b1519212f45a2a7404860bc059f3e'

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## Imports

In [4]:
import pandas as pd
import torch
import pyterrier as pt
from model.Bert import Bert
from model.Train import Trainer
from model.Loss import LSmoothing
from model.DataManager import DataManager as DM
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [11]:
import os
os.environ["JAVA_HOME"] = "C:/Program Files/Java/jdk-21"
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"], mem=8000)

## Variables

In [12]:
nb_labels = 2
epochs = 100
learning_rate = 1e-4
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


## Loading Data

In [13]:
# !rm -rf cord19-index

In [14]:
batch_size = 16

In [24]:
dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')
indexer = pt.index.IterDictIndexer('./cord19-index')
indexref = indexer.index(dataset.get_corpus_iter(), fields=('title', 'abstract'))
index = pt.IndexFactory.of(indexref)

18:01:10.922 [main] ERROR org.terrier.structures.Index - Cannot create new index: path c:\Cours-Sorbonne\S2\RITAL\TME_RI\PROJET\.\var\cord19-index does not exist, or cannot be written to


JavaException: JVM exception occurred: Cannot create new index: path c:\Cours-Sorbonne\S2\RITAL\TME_RI\PROJET\.\var\cord19-index does not exist, or cannot be written to java.lang.IllegalArgumentException

In [ ]:
def filter_qrels(qrels, topics):
    return qrels[qrels['qid'].isin(topics['qid'])]

topics = dataset.get_topics('title')
qrels = dataset.get_qrels()

train_topics, test_topics = train_test_split(topics, test_size=0.2, random_state=42)
val_topics, test_topics = train_test_split(test_topics, test_size=0.5, random_state=42)

train_qrels = filter_qrels(qrels, train_topics)
val_qrels = filter_qrels(qrels, val_topics)
test_qrels = filter_qrels(qrels, test_topics)

print(train_topics.shape, test_topics.shape, val_topics.shape)
print(train_qrels.shape, val_qrels.shape, test_qrels.shape)

(40, 2) (5, 2) (5, 2)
(56292, 4) (6417, 4) (6609, 4)


In [ ]:
def filter_doc_rel(topic,rel):
  df = pd.merge(topic, rel, on='qid')
  relevant_docs = df[df['label'] == 1]
  non_relevant_docs = df[df['label'] == 0]
  # Filtrer pour ne garder qu'un seul document pertinent par requête
  filtered_relevant_docs = relevant_docs.drop_duplicates(subset='qid', keep='first')
  # Combiner les documents filtrés pertinents et non pertinents
  final_df = pd.concat([filtered_relevant_docs, non_relevant_docs])
  # Réordonner le DataFrame par 'qid' pour la clarté
  final_df = final_df.sort_values(by='qid').reset_index(drop=True)
  return final_df.drop(columns=["qid","iteration"])

train_df = filter_doc_rel(train_topics, train_qrels)
test_df = filter_doc_rel(test_topics, test_qrels)
val_df = filter_doc_rel(val_topics, val_qrels)

print(train_df.shape, test_df.shape, val_df.shape)

(34828, 3) (4347, 3) (3527, 3)


In [ ]:
train_inputs, train_masks, train_labels = DM.prepare_data(train_df)
val_inputs, val_masks, val_labels = DM.prepare_data(val_df)
test_inputs, test_masks, test_labels = DM.prepare_data(test_df)

In [ ]:
train_dataset = DM(train_inputs, train_masks, train_labels)
val_dataset = DM(val_inputs, val_masks, val_labels)
test_dataset = DM(test_inputs, test_masks, test_labels)

train_loader = DataLoader(
    train_dataset,
    sampler=RandomSampler(train_dataset),
    batch_size=batch_size
)

val_loader = DataLoader(
    val_dataset,
    sampler=SequentialSampler(val_dataset),
    batch_size=batch_size
)

test_loader = DataLoader(
    test_dataset,
    sampler=SequentialSampler(test_dataset),
    batch_size=batch_size
)

## Training

In [ ]:
epochs = 10
learning_rate = 1e-3

In [ ]:
model = Bert()
model = model.to(device)
optimizer = torch.optim.AdamW
loss = LSmoothing()
trainer = Trainer()
history = trainer.set_model(model)\
    .set_loader(train_loader, val_loader, test_loader)\
    .set_loss_fn(loss)\
    .set_optimizer(optimizer)\
    .fit(learning_rate, epochs)

TypeError: super(type, obj): obj must be an instance or subtype of type